In [2]:
import numpy as np
from BPnumba.NumAG import createAG
from BPmodule.BPGen import PROBLEM2, PROBLEM3, PROBLEM4, PROBLEM5
from BPnumba.GeneticOperators import CreatePoblation, InstancePob,CreateHeuristicPob,Ind
from BPnumba.DFFA import createDFFA
from BPnumba.ABCnum import createDABC
from numba import njit, objmode, prange
import time
from numba.typed import List as NumbaList
import matplotlib.pyplot as plt
from typing import List
from numpy import savetxt


In [3]:
P2A1 = PROBLEM2[0]
P2A2 = PROBLEM2[1]
P3A1 = PROBLEM3[0]
P3A2 = PROBLEM3[1]
P4A1 = PROBLEM4[0]
P4A2 = PROBLEM4[1]
P5A1 = PROBLEM5[0]
P5A2 = PROBLEM5[1]
numTest = 20
n = 4
def zerovec(y, x): return np.zeros(shape=[x, y], dtype=np.float64)

createText = lambda x,y : savetxt(
    fname = x + ".csv",
    X= y,
    delimiter=", ",
    header=" Fi BoxesPacked MaxItr Time(s) | 50 individuos",
    fmt='%g'
)

ag2 = zerovec(n, numTest)
ab2 = zerovec(n, numTest)
fa2 = zerovec(n, numTest)
ag3 = zerovec(n, numTest)
ab3 = zerovec(n, numTest)
fa3 = zerovec(n, numTest)
ag4 = zerovec(n, numTest)
ab4 = zerovec(n, numTest)
fa4 = zerovec(n, numTest)
ag5 = zerovec(n, numTest)
ab5 = zerovec(n, numTest)
fa5 = zerovec(n, numTest)


In [4]:
@njit(parallel=True)
def Test(num_test: int, ag, sameInstance: List[int], maxItr: int, agL: List[List[float]], datos: List[List[List[int]]], contenedor: List[int]):
    n = len(datos)
    for _ in prange(num_test):
        Pob = InstancePob(sameInstance, datos, contenedor)
        with objmode(time1='f8'):
            time1 = time.perf_counter()
        bestInd:Ind = ag.Train(maxItr, Pob, datos, contenedor)
        epochs = len(ag.bestfi)
        with objmode(last='f8'):
            last = time.perf_counter() - time1
        agL[_] = np.array([bestInd.fi, n-bestInd.load,
                          epochs, last], dtype=np.float64)


In [5]:
pr_selec = 0.85
pr_cross = 0.75
pr_mut = 0.2
gamma = 1/(n*n)
pop =50
maxItr=1000

In [6]:
contenedor = P3A1[0][0]
datos = P3A1[1]
n = len(datos)
idBoxes = np.arange(1, n+1)
sameInstance = CreatePoblation(pop, idBoxes)

In [7]:
ag = createAG(pr_selec, pr_cross, pr_mut)
Test(numTest, ag,sameInstance, maxItr, ag3, datos, contenedor)

In [ ]:
abc = createDABC(pop,n)
Test(numTest, abc,sameInstance,maxItr, ab3, datos, contenedor)

In [ ]:
dffa = createDFFA(gamma)
Test(numTest, dffa,sameInstance, maxItr, fa3, datos, contenedor)

In [ ]:
createText("FFAp3",fa3)
createText("AGp3",ag3)
createText("ABCp3",ab3)

In [ ]:
contenedor = P4A1[0][0]
datos = P4A1[1]
n = len(datos)
idBoxes = np.arange(1, n+1)
sameInstance = CreatePoblation(pop, idBoxes)

In [ ]:
ag = createAG(pr_selec, pr_cross, pr_mut)
Test(numTest, ag,sameInstance, maxItr, ag4, datos, contenedor)

In [ ]:
abc = createDABC(pop,n)
Test(numTest, abc,sameInstance,maxItr, ab4, datos, contenedor)

In [ ]:
dffa = createDFFA(gamma)
Test(numTest, dffa,sameInstance, maxItr, fa4, datos, contenedor)

In [ ]:
createText("FFAp4",fa4)
createText("AGp4",ag4)
createText("ABCp4",ab4)